# CommonLit Readability (Kaggle Challenge)
## Training with BERT

In [1]:
import os 
import pandas as pd
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import (
    Dataset,
    DataLoader
)
from transformers import (
    AutoTokenizer,
    BertForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup
)

from utils.train_with_bert import BERTDataset, training, validating

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Preprocess data

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# get max length of input ids in train_df
text_length = list()
for text in train_df["excerpt"]:
    tokens = tokenizer.encode(text)
    text_length.append(len(tokens))

max_length = max(text_length)
print(max_length)

314


### Splitting the dataset

In [9]:
train_df["kfold"] = train_df.index % 5

train_fold = train_df[train_df["kfold"] != 0].reset_index(drop=True)
valid_fold = train_df[train_df["kfold"] == 0].reset_index(drop=True)

### Building Dataset and DataLoader

In [10]:
train_ds = BERTDataset(tokenizer, max_length, train_fold["excerpt"], train_fold["target"])
valid_ds = BERTDataset(tokenizer, max_length, valid_fold["excerpt"], valid_fold["target"])

train_bs = 8
valid_bs = 16

train_dl = DataLoader(
    train_ds, 
    batch_size=train_bs,
    shuffle=True,
    num_workers=2,
    pin_memory=False
)
valid_dl = DataLoader(
    valid_ds, 
    batch_size=valid_bs,
    shuffle=False,
    num_workers=2,
    pin_memory=False
)

### Model

In [ ]:
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

model.to(device)
model.train()

### Training function

In [7]:
optimizer = AdamW(model.parameters(), lr=0.005, weight_decay=0.001)

epochs = 20
train_steps = int(len(train_fold)/train_bs*epochs)
num_steps = int(train_steps * 0.1)

scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)

In [ ]:
train_losses = list()
valid_losses = list()

for epoch in tqdm(range(epochs)):
    
    print("EPOCH", epoch + 1)
    
    train_loss = training(train_dl, model, optimizer, scheduler, device)
    print(f"Training Loss: {train_loss:.4f}")
    
    valid_loss = validating(valid_dl, model, device)
    print(f"Validation Loss: {valid_loss:.4f}")
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)